<a href="https://colab.research.google.com/github/JimmyCh2001/kkcompany_datagame_schubert/blob/068_branch/kkcompany_datagame_schubert_019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Get Data from Kaggle

In [3]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c datagame-2023

 99% 398M/401M [00:06<00:00, 64.3MB/s]
100% 401M/401M [00:06<00:00, 66.2MB/s]


In [4]:
!unzip datagame-2023

Archive:  datagame-2023.zip
  inflating: label_test_source.parquet  
  inflating: label_train_source.parquet  
  inflating: label_train_target.parquet  
  inflating: meta_song.parquet       
  inflating: meta_song_composer.parquet  
  inflating: meta_song_genre.parquet  
  inflating: meta_song_lyricist.parquet  
  inflating: meta_song_producer.parquet  
  inflating: meta_song_titletext.parquet  
  inflating: sample.csv              


#Import Library

In [21]:
import pandas as pd
import numpy as np

#Data Preprocessing

Load data from parquet file to code

In [26]:
label_test = pd.read_parquet("/content/label_test_source.parquet")
label_train = pd.read_parquet("/content/label_train_source.parquet")
label_target = pd.read_parquet("/content/label_train_target.parquet")
meta_song = pd.read_parquet("/content/meta_song.parquet")
meta_song_composer = pd.read_parquet("/content/meta_song_composer.parquet")
meta_song_genre = pd.read_parquet("/content/meta_song_genre.parquet")
meta_song_lyricist = pd.read_parquet("/content/meta_song_lyricist.parquet")
meta_song_producer = pd.read_parquet("/content/meta_song_producer.parquet")
meta_song_title = pd.read_parquet("/content/meta_song_titletext.parquet")

In [14]:
meta_song.head()

,song_id,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06,3211cf51bfa3afff4f264a110212f615,ce4db56f6a48426643b08038139a8a75,c1079ef109db2aba72f78c632ab73803,3211cf51bfa3afff4f264a110212f615,086136ec34f3ff6eb89cfa7c446bd69f
1,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06,3211cf51bfa3afff4f264a110212f615,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,3211cf51bfa3afff4f264a110212f615,086136ec34f3ff6eb89cfa7c446bd69f
2,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,de047ee6a48ebb4b17c93ef6ad6bbe75,79de4f094a315dce9cea618d57b4068e
3,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,de047ee6a48ebb4b17c93ef6ad6bbe75,bec1278aaffb1f3df911af327ef8e9c5
4,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,05b3fe573355f70f606b3ad063c45da8,79de4f094a315dce9cea618d57b4068e


In [5]:
meta_song_title.head()

,song_id,title_text_id
0,40985b01760119571bb9d9d6a44e003d,9cc814abf8f2edfe616f36934986c81b
1,67e88ec980c60a4774243f12a6505d8c,51da6f7499addd8acb161d03cb7b87a0
2,1f3b644b4aa4f0a6273c22711e585494,c1079ef109db2aba72f78c632ab73803
3,948db75f718d788e4da02a365b4673af,c1079ef109db2aba72f78c632ab73803
4,00f259930c0454a942bb45a63620b639,bafe4ea3c99161d13843b345569a6d1a


In [32]:
label_train

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,751,6027767fad949f3ca5e772df04924949,1659598909,1,7,1
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1,7,2
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0,7,3
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0,7,4
4,751,8b32f88104ecf859be934d9b45f30cd1,1659599218,1,7,5
...,...,...,...,...,...,...
11445175,458622,3ba8c1e455e27172edec64deff3a8b15,1662339887,2,7,16
11445176,458622,3ba8c1e455e27172edec64deff3a8b15,1662340090,2,7,17
11445177,458622,3ba8c1e455e27172edec64deff3a8b15,1662340294,2,7,18
11445178,458622,3ba8c1e455e27172edec64deff3a8b15,1662340497,2,7,19


from matplotlib import pyplot as plt
label_train['session_id'].plot(kind='hist', bins=20, title='session_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
label_train['unix_played_at'].plot(kind='hist', bins=20, title='unix_played_at')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
label_train['play_status'].plot(kind='hist', bins=20, title='play_status')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
label_train['login_type'].plot(kind='hist', bins=20, title='login_type')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
label_train.plot(kind='scatter', x='login_type', y='listening_order', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
label_train['session_id'].plot(kind='line', figsize=(8, 4), title='session_id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
label_train['unix_played_at'].plot(kind='line', figsize=(8, 4), title='unix_played_at')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
label_train['play_status'].plot(kind='line', figsize=(8, 4), title='play_status')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
label_train['login_type'].plot(kind='line', figsize=(8, 4), title='login_type')
plt.gca().spines[['top', 'right']].set_visible(False)

In [31]:
label_test

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,598,43519381e8351d7c6afcc66bcc0a0636,1660177283,1,7,1
1,598,928e2e8e316f29efd986fa8d23429431,1660177503,1,7,2
2,598,5229859624d3b58c35646154cf366bf4,1660177582,1,7,3
3,598,49e50187f7a7f658623807780f499a15,1660177584,1,7,4
4,598,bc8b660fe4d4bfa0a13334beae0cb750,1660177587,1,7,5
...,...,...,...,...,...,...
2861275,714498,13d62012feb791ded330b17098230e0d,1666147178,0,7,16
2861276,714498,a3f4cb9a06f48f2e9ea381be3afc64b3,1666147262,0,7,17
2861277,714498,9bb5e3b8b43fe1e306459c89ab263823,1666147366,0,7,18
2861278,714498,e1ec04cca3d78e9e7fd9793ae45d3638,1666147464,0,7,19


In [30]:
label_target

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,307,75c2aa348888f982d85e3f870e6ba5b2,1659942029,1,8,21
1,307,0cab8863e5440551c7b37e59635ec18e,1659942032,1,8,22
2,307,4d5aceee5c9731151ca69f0946ffa71f,1659942175,1,8,23
3,307,929b07d69451684f4f0f6e3bcc2a62d6,1659942272,0,8,24
4,307,12ae4e616d3e5c7bd53ec771797f596b,1659942277,1,8,25
...,...,...,...,...,...,...
2861290,714580,55728030c86e1afb7aa4a31fee56d3b8,1666643323,0,7,21
2861291,714580,741590eee67a948a1b8b3a9ff60682bf,1666643442,0,7,22
2861292,714580,3c90623f4a5995b59a4f4d4eafcde4cc,1666643515,0,7,23
2861293,714580,4a3c3bf3793672422bc87ee280087315,1666643581,0,7,24


Merge Meta Data to one variabe

In [8]:
list_meta = [meta_song_composer, meta_song_genre, meta_song_title, meta_song_lyricist, meta_song_producer]

for meta in list_meta:
  meta_song = pd.merge(meta_song, meta, on="song_id", how="inner")

meta_song.head()

,song_id,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,title_text_id,lyricist_id,producer_id
0,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06,3211cf51bfa3afff4f264a110212f615,ce4db56f6a48426643b08038139a8a75,c1079ef109db2aba72f78c632ab73803,3211cf51bfa3afff4f264a110212f615,086136ec34f3ff6eb89cfa7c446bd69f
1,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06,3211cf51bfa3afff4f264a110212f615,b856b6781d370a3645c6dde0c20b3597,c1079ef109db2aba72f78c632ab73803,3211cf51bfa3afff4f264a110212f615,086136ec34f3ff6eb89cfa7c446bd69f
2,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,de047ee6a48ebb4b17c93ef6ad6bbe75,79de4f094a315dce9cea618d57b4068e
3,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,de047ee6a48ebb4b17c93ef6ad6bbe75,bec1278aaffb1f3df911af327ef8e9c5
4,f83417fa016c2d7fbc90c5ccc9068817,2362.0,189.0,52394.0,62.0,1956-05,f768fe07b6a0d080f32e4d4633f00755,b856b6781d370a3645c6dde0c20b3597,d96c30559e54471ff893bd50a28b421e,05b3fe573355f70f606b3ad063c45da8,79de4f094a315dce9cea618d57b4068e


Check Data Type of each columns in **meta_song**

In [19]:
column_data_types = meta_song.dtypes

print("Data Types of Each Column:")
print(column_data_types)

Data Types of Each Column:
song_id           object
artist_id        float64
song_length      float64
album_id         float64
language_id      float64
album_month       object
composer_id       object
genre_id          object
title_text_id     object
lyricist_id       object
producer_id       object
dtype: object


In [33]:
#Drop those record that had the play_status = 0
label_train_dropped = label_train.replace(-1, np.nan).dropna()
label_test_dropped = label_test.replace(-1, np.nan).dropna()
label_train_dropped
label_test_dropped

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,598,43519381e8351d7c6afcc66bcc0a0636,1660177283,1.0,7,1
1,598,928e2e8e316f29efd986fa8d23429431,1660177503,1.0,7,2
2,598,5229859624d3b58c35646154cf366bf4,1660177582,1.0,7,3
3,598,49e50187f7a7f658623807780f499a15,1660177584,1.0,7,4
4,598,bc8b660fe4d4bfa0a13334beae0cb750,1660177587,1.0,7,5
...,...,...,...,...,...,...
2861275,714498,13d62012feb791ded330b17098230e0d,1666147178,0.0,7,16
2861276,714498,a3f4cb9a06f48f2e9ea381be3afc64b3,1666147262,0.0,7,17
2861277,714498,9bb5e3b8b43fe1e306459c89ab263823,1666147366,0.0,7,18
2861278,714498,e1ec04cca3d78e9e7fd9793ae45d3638,1666147464,0.0,7,19
